# STA 663 Project 1: 
- Huggingface: mastergopote44/Long-Term-Care-Aggregated-Data
- Name: Justin Kao
- School: Duke University

## Load dataset from SOA website

In [56]:
import requests
from io import BytesIO
from zipfile import ZipFile
import pandas as pd

# Directory to store the extracted files
desired_directory = '/Users/justinkao/Desktop/Durham NC/Duke University/Courses/S_2024_STA 663 Statistical Computing and Computation(Dr. Ouwen Huang)/Project 1/STA663_Project_1'  # Replace with your actual path

# URLs of the zip files
zip_files = {
    'incidence': 'https://www.soa.org/4a33aa/globalassets/assets/files/resources/experience-studies/2020/2000-2016-ltc-incidence.zip',
    'termination': 'https://www.soa.org/4a2e5d/globalassets/assets/files/resources/experience-studies/2020/2000-2016-ltc-termination.zip'
}

# DataFrames dictionary
dataframes = {}

# Download and unzip the files
for name, zip_url in zip_files.items():
    response = requests.get(zip_url)
    # Check if the request was successful
    if response.ok:
        with ZipFile(BytesIO(response.content)) as thezip:
            # Extract all files from the zip into a directory
            thezip.extractall(desired_directory)
            # Loop through each file in the zip
            for zipinfo in thezip.infolist():
                # Construct the full path to the extracted file
                file_path = f'{desired_directory}/{zipinfo.filename}'
                # Assuming the file is a tab-separated txt file
                df = pd.read_csv(file_path, sep='\t', low_memory=False)  # Adjust sep if necessary
                # Assign the DataFrame to the corresponding variable
                dataframes[name] = df
    else:
        print(f"Failed to retrieve {zip_url}")

In [ ]:
incidence_df = dataframes['incidence']
incidence_df

In [ ]:
termination_df = dataframes['termination']
termination_df

## Inspect for the presence of outliers and missing values in the variable

#### 1. Check out the dataframe at first and make sure the unique vlaue of each variable is reasonable 

In [ ]:
unique_values = {col: incidence_df[col].unique() for col in incidence_df.columns}
unique_values

#### 2. Checking for "Unknown" Values That Do Not Contribute Information
The dataset contains 627,190 rows and 13 columns. For actuarial analysis, it is essential that the variables 'Issue_Age_Bucket', 'Incurred_Age_Bucket', and 'Issue_Year_Bucket' contain valid information since they are crucial in evaluating claim incidences. Rows with 'Unknown' values in these variables do not contribute to the analysis and can potentially skew the results. Currently, there are 519 'Unknown' values in 'Issue_Age_Bucket', 481 in 'Incurred_Age_Bucket', and 431 in 'Issue_Year_Bucket'. Removing rows with 'Unknown' values in these three variables is a necessary step to ensure data quality and reliability of the analysis.


In [26]:
# Calculate the number of 'Unknown' values in the 'Issue_Age_Bucket' column
unknown_issue_age = (incidence_df['Issue_Age_Bucket'] == 'Unknown').sum()
print(f"Number of 'Unknown' values in 'Issue_Age_Bucket': {unknown_issue_age}")

Number of 'Unknown' values in 'Issue_Age_Bucket': 519


In [77]:
unknown_incurred_age = (incidence_df['Incurred_Age_Bucket'] == 'Unknown').sum()
print(f"Number of 'Unknown' values in 'Incurred_Age_Bucket': {unknown_incurred_age}")


Number of 'Unknown' values in 'Incurred_Age_Bucket': 481


In [76]:
unknown_issue_year = (incidence_df['Issue_Year_Bucket'] == 'Unknown').sum()
print(f"Number of 'Unknown' values in 'Issue_Year_Bucket': {unknown_issue_year}")

Number of 'Unknown' values in 'Issue_Year_Bucket': 431


#### 3. Eliminate rows where below three variables have 'Unknown' values
- Counts of 'Unknown' values in key variables:
- 'Issue_Age_Bucket': 519
- 'Incurred_Age_Bucket': 481
- 'Issue_Year_Bucket': 431


In [78]:
# Eliminate rows where these three variables have 'Unknown' values
filtered_incidence_df = incidence_df[
    (incidence_df['Issue_Age_Bucket'] != 'Unknown') & 
    (incidence_df['Incurred_Age_Bucket'] != 'Unknown') & 
    (incidence_df['Issue_Year_Bucket'] != 'Unknown')
]

# Display the shape of the dataframe after elimination
print(f"Dataframe shape after elimination: {filtered_incidence_df.shape}")

Dataframe shape after elimination: (2088757, 31)


In [ ]:
filtered_incidence_df

#### 4. Inspect Underwriting_Type with Unknown value

In [41]:
unknown_Underwriting_Type = (incidence_df['Underwriting_Type'] == 'Unknown').sum()
print(f"Number of 'Unknown' values in 'Region': {}")

Number of 'Unknown' values in 'Region': 465920


#### 5. Inspect Region with Unknown value

In [40]:
unknown_Region = (incidence_df['Region'] == 'Unknown').sum()
print(f"Number of 'Unknown' values in 'Region': {unknown_Region}")

Number of 'Unknown' values in 'Region': 388271


## Impute Region with Conditional Mode Imputation

In [80]:
# Assuming 'filtered_incidence_df' is your DataFrame
# Example: Using 'Policy_Year' as the related variable for demonstration

# Step 1: Replace 'Unknown' with np.nan for easier handling with pandas methods
filtered_incidence_df['Region'].replace('Unknown', np.nan, inplace=True)

# Step 2 & 3: Impute 'Unknown' (now np.nan) in 'Region' based on the mode of each 'Policy_Year' group
for policy_year in filtered_incidence_df['Policy_Year'].unique():
    # Compute the mode of 'Region' for the current 'Policy_Year'
    mode_region = filtered_incidence_df.loc[filtered_incidence_df['Policy_Year'] == policy_year, 'Region'].mode()
    if not mode_region.empty:
        # Impute 'Unknown' values with the mode for this subset
        filtered_incidence_df.loc[(filtered_incidence_df['Policy_Year'] == policy_year) & (filtered_incidence_df['Region'].isna()), 'Region'] = mode_region[0]

# If there are still any 'Unknown' (np.nan) values left, fill them with the overall mode
overall_mode = filtered_incidence_df['Region'].mode()[0]
filtered_incidence_df['Region'].fillna(overall_mode, inplace=True)

# Check the imputation result
print(filtered_incidence_df['Region'].value_counts())


/var/folders/fs/5ct03tj50v1_hbz7dk31c6mw0000gn/T/ipykernel_13389/1088997212.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_incidence_df['Region'].replace('Unknown', np.nan, inplace=True)


Region
03: South        933136
01: Mid-West     476202
04: West         416651
02: Northeast    257988
05: Other          4780
Name: count, dtype: int64


/var/folders/fs/5ct03tj50v1_hbz7dk31c6mw0000gn/T/ipykernel_13389/1088997212.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_incidence_df['Region'].fillna(overall_mode, inplace=True)


In [81]:
filtered_incidence_df

,Group_Indicator,Gender,Issue_Age_Bucket,Incurred_Age_Bucket,Issue_Year_Bucket,Policy_Year,Marital_Status,Premium_Class,Underwriting_Type,Coverage_Type_Bucket,...,ALF_EP_Bucket,HHC_EP_Bucket,Region,Active_Exposure,Total_Exposure,Claim_Count,Count_NH,Count_ALF,Count_HHC,Count_Unk
0,Group,Female,55-59,55-59,2003-2005,1-3 years,Single,Standard,Other,Comprehensive,...,0,0,03: South,37.583332,37.583332,0,0,0,0,0
1,Individual,Female,60-64,70-74,1997-1999,10-12 years,Single,Preferred,Other,Comprehensive,...,0,0,02: Northeast,14.500000,14.500000,0,0,0,0,0
2,Group,Female,50-54,60-64,2000-2002,10-12 years,Unknown,Standard,Unknown,Comprehensive,...,0,0,03: South,2250.000000,2252.000000,0,0,0,0,0
3,Individual,Female,55-59,70-74,1994-1996,13-15 years,Married,Standard,Other,Comprehensive,...,0,0,01: Mid-West,19.000000,19.000000,0,0,0,0,0
4,Individual,Male,60-64,65-69,2003-2005,4-6 years,Married,Substandard,Full underwriting,Comprehensive,...,90,20,02: Northeast,2.000000,2.000000,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2089318,Individual,Male,55-59,60-64,2012-2014,1-3 years,Married,Substandard,Full underwriting,Comprehensive,...,0,0,01: Mid-West,7.083333,7.083333,0,0,0,0,0
2089319,Individual,Male,65-69,70-74,1997-1999,7-9 years,Married,Standard,Full underwriting,Comprehensive,...,180,180,02: Northeast,2.000000,2.000000,0,0,0,0,0
2089320,Individual,Female,65-69,75-79,1994-1996,7-9 years,Single,Preferred,Other,Comprehensive,...,0,0,01: Mid-West,6.000000,6.000000,0,0,0,0,0
2089321,Individual,Female,50-54,55-59,1994-1996,7-9 years,Single,Standard,Unknown,Comprehensive,...,0,0,03: South,6.000000,6.000000,0,0,0,0,0


## Split both "Incidence" and "Termination" datasets into train/test

In [82]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Now we will split the 'incidence' DataFrame into training and validation sets
train__filtered_incidence_df, validation_filtered_incidence_df = train_test_split(filtered_incidence_df, test_size=0.2, random_state=42)

# train_df is now your training set, and validation_df is your validation set.

In [85]:
train__filtered_incidence_df

,Group_Indicator,Gender,Issue_Age_Bucket,Incurred_Age_Bucket,Issue_Year_Bucket,Policy_Year,Marital_Status,Premium_Class,Underwriting_Type,Coverage_Type_Bucket,...,ALF_EP_Bucket,HHC_EP_Bucket,Region,Active_Exposure,Total_Exposure,Claim_Count,Count_NH,Count_ALF,Count_HHC,Count_Unk
1702737,Individual,Female,50-54,60-64,2003-2005,7-9 years,Single,Preferred,Other,Comprehensive,...,90,0,04: West,19.583332,19.583332,0,0,0,0,0
1380059,Individual,Female,70-74,80-84,2000-2002,13-15 years,Married,Standard,Unknown,Comprehensive,...,0,0,03: South,84.333336,108.916660,8,0,0,8,0
837211,Individual,Female,60-64,60-64,2000-2002,4-6 years,Unknown,Standard,Unknown,Comprehensive,...,90,0,03: South,3.000000,3.000000,0,0,0,0,0
1315575,Individual,Female,80-84,80-84,1994-1996,4-6 years,Married,Standard,Full underwriting,Other,...,0,0,03: South,3.750000,3.750000,0,0,0,0,0
907204,Individual,Female,55-59,55-59,1994-1996,4-6 years,Married,Standard,Full underwriting,Other,...,30,0,03: South,2.583333,2.583333,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259245,Group,Female,65-69,65-69,2003-2005,4-6 years,Married,Standard,Other,Comprehensive,...,0,0,02: Northeast,4.000000,4.000000,0,0,0,0,0
1414796,Individual,Female,55-59,65-69,1997-1999,10-12 years,Single,Standard,Unknown,Comprehensive,...,0,0,04: West,62.750000,62.833332,1,0,0,1,0
131965,Individual,Male,70-74,80-84,2009-2011,7-9 years,Unknown,Standard,Full underwriting,Comprehensive,...,0,0,02: Northeast,2.416666,2.416666,0,0,0,0,0
671338,Individual,Female,70-74,85-89,1997-1999,15+ years,Single,Preferred,Unknown,Comprehensive,...,0,0,03: South,14.249999,19.916664,3,0,1,2,0


In [84]:
validation_filtered_incidence_df

,Group_Indicator,Gender,Issue_Age_Bucket,Incurred_Age_Bucket,Issue_Year_Bucket,Policy_Year,Marital_Status,Premium_Class,Underwriting_Type,Coverage_Type_Bucket,...,ALF_EP_Bucket,HHC_EP_Bucket,Region,Active_Exposure,Total_Exposure,Claim_Count,Count_NH,Count_ALF,Count_HHC,Count_Unk
1756216,Group,Female,50-54,55-59,2009-2011,7-9 years,Unknown,Standard,Unknown,Comprehensive,...,0,0,03: South,32.749996,32.749996,0,0,0,0,0
2001415,Individual,Female,60-64,75-79,1994-1996,15+ years,Married,Standard,Full underwriting,Comprehensive,...,0,0,02: Northeast,2.000000,2.000000,0,0,0,0,0
454164,Group,Male,0-49,0-49,1997-1999,10-12 years,Married,Standard,Other,Comprehensive,...,0,0,02: Northeast,7.000000,7.000000,0,0,0,0,0
1073250,Individual,Male,65-69,80-84,1997-1999,13-15 years,Married,Standard,Other,Comprehensive,...,0,0,01: Mid-West,63.583332,65.583328,3,1,1,1,0
811617,Individual,Male,65-69,65-69,1994-1996,4-6 years,Married,Standard,Unknown,Comprehensive,...,0,0,02: Northeast,23.416662,23.416662,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736427,Individual,Female,60-64,75-79,2000-2002,15+ years,Married,Preferred,Other,Comprehensive,...,90,30,01: Mid-West,1.500000,1.500000,0,0,0,0,0
506473,Individual,Female,65-69,65-69,2000-2002,4-6 years,Unknown,Substandard,Full underwriting,Other,...,0,0,01: Mid-West,3.000000,3.000000,0,0,0,0,0
1894427,Individual,Female,0-49,60-64,2000-2002,15+ years,Single,Preferred,Unknown,Comprehensive,...,0,0,03: South,2.083333,2.083333,0,0,0,0,0
244198,Group,Female,55-59,60-64,2006-2008,1-3 years,Unknown,Standard,Full underwriting,Comprehensive,...,0,0,03: South,2.000000,2.000000,0,0,0,0,0


In [5]:
# Now we will split the 'incidence' DataFrame into training and validation sets

train_termination_df, validation_termination_df = train_test_split(termination_df, test_size=0.2, random_state=42)